# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GNNTrainer** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [ ]:
from gnn import GNNTrainer
from dataset_preprocess import (
    MovieLensLoader,
    PROMPT_KGE_DIMENSION,
    EMBEDDING_KGE_DIMENSION,
)
from llm import (
    PromptBertClassifier,
    VanillaBertClassifier,
    EmbeddingsBertClassifierBase,
)

In [ ]:
EPOCHS = 20

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [ ]:
loader = MovieLensLoader()

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-latest-small.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [ ]:
loader.data

Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [ ]:
gnn_trainer_prompt = GNNTrainer(
    loader.data,
    loader.gnn_train_data,
    loader.gnn_val_data,
    loader.gnn_test_data,
    kge_dimension=PROMPT_KGE_DIMENSION,
)
gnn_trainer_embedding = GNNTrainer(
    loader.data,
    loader.gnn_train_data,
    loader.gnn_val_data,
    loader.gnn_test_data,
    hidden_channels=EMBEDDING_KGE_DIMENSION,
    kge_dimension=EMBEDDING_KGE_DIMENSION,
)

We then train and validate the model on the link prediction task.

If the model is already trained, we can skip this part.
Training the models can take up to 5 minutes.

In [ ]:
print("Prompt Training")
gnn_trainer_prompt.train_model(loader.gnn_train_data, EPOCHS)
gnn_trainer_prompt.validate_model(loader.gnn_test_data)
print("Embedding Training")
gnn_trainer_embedding.train_model(loader.gnn_train_data, EPOCHS)
gnn_trainer_embedding.validate_model(loader.gnn_test_data)


Next we produce the KGEs for every edge in the dataset. These embeddings can then be used for the LLM on the link-prediction task.

In [ ]:
prompt_embeddings = gnn_trainer_prompt.get_saved_embeddings("prompt")
embedding_embeddings = gnn_trainer_embedding.get_saved_embeddings("embedding")
save = False
if prompt_embeddings is None or embedding_embeddings is None:
    prompt_embeddings = gnn_trainer_prompt.generate_embeddings(loader.llm_df)
    embedding_embeddings = gnn_trainer_embedding.generate_embeddings(loader.llm_df)
    save = True

loader.append_prompt_graph_embeddings(prompt_embeddings, save=save)
loader.append_embedding_graph_embeddings(embedding_embeddings, save=save)


Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [ ]:
vanilla_bert_classifier = VanillaBertClassifier(
    loader.llm_df, loader.source_df, loader.target_df
)

Next we generate a vanilla llm dataset and tokenize it for training.

In [ ]:
dataset_vanilla = loader.generate_vanilla_dataset(
    vanilla_bert_classifier.tokenize_function
)

Next we train the model on the produced dataset. This can be skipped, if already trained ones.

In [ ]:
vanilla_bert_classifier.train_model_on_data(dataset_vanilla, epochs=EPOCHS)

Next we initialize the prompt encoder only classifier. This classifier uses the vanilla prompt and the KGEs for its link prediction.

In [ ]:
prompt_bert_classifier = PromptBertClassifier(
    loader,
    gnn_trainer_prompt.get_embedding,
    model_max_length=512,
)

We also generate a prompt dataset, this time the prompts also include the KGEs.

In [ ]:
dataset_prompt = loader.generate_prompt_embedding_dataset(
    prompt_bert_classifier.tokenize_function,
)

We also train the model. This can be skipped if already done ones.

In [ ]:
prompt_bert_classifier.train_model_on_data(dataset_prompt, epochs=EPOCHS)

In [ ]:
embedding_bert_only_classifier = EmbeddingsBertClassifierBase(
    loader,
    gnn_trainer_embedding.get_embedding,
)

In [ ]:
dataset_embedding = loader.generate_embedding_dataset(
    embedding_bert_only_classifier.tokenizer.sep_token,
    embedding_bert_only_classifier.tokenizer.pad_token,
    embedding_bert_only_classifier.tokenize_function,
)

In [ ]:
embedding_bert_only_classifier.train_model_on_data(dataset_embedding, epochs=EPOCHS)

In [ ]:
loader.add_false_edges(
    2.0, gnn_trainer_prompt.get_embedding, gnn_trainer_embedding.get_embedding
)

In [ ]:
vanilla_bert_classifier = VanillaBertClassifier(
    loader.llm_df,
    loader.source_df,
    loader.target_df,
    false_ratio=-1,
)
prompt_bert_classifier = PromptBertClassifier(
    loader,
    gnn_trainer_prompt.get_embedding,
    model_max_length=512,
    false_ratio=-1,
)
embedding_bert_only_classifier = EmbeddingsBertClassifierBase(
    loader,
    gnn_trainer_embedding.get_embedding,
    false_ratio=-1,
)


In [ ]:
dataset_vanilla_fixed = loader.generate_vanilla_dataset(
    vanilla_bert_classifier.tokenize_function, suffix="_fixed"
)
dataset_prompt_fixed = loader.generate_prompt_embedding_dataset(
    prompt_bert_classifier.tokenize_function,
    suffix="_fixed",
)
dataset_embedding_fixed = loader.generate_embedding_dataset(
    embedding_bert_only_classifier.tokenizer.sep_token,
    embedding_bert_only_classifier.tokenizer.pad_token,
    embedding_bert_only_classifier.tokenize_function,
    suffix="_fixed",
)

In [ ]:
vanilla_bert_classifier.forward_dataset_and_save_outputs(
    dataset_vanilla_fixed, epochs=1
)
prompt_bert_classifier.forward_dataset_and_save_outputs(dataset_prompt_fixed, epochs=1)
df = embedding_bert_only_classifier.forward_dataset_and_save_outputs(
    dataset_embedding_fixed, epochs=1
)